In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [41]:
from os import listdir
from os.path import isfile, join

mypath = '/Users/data/Documents/actas/datos/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [42]:
#files

In [43]:
# quiero saber cuáles son todos los campos posibles para hacer columnas en el df

# agrego estos a mano porque en el loop no salen solos
campos = ['id', 'documento', 'pagina', 'completed_by']
for f in files:
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        
        for i in range(len(ground_truth)):
            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                # acá salteo todos los que no tengan el formato que me interesa
                try:
                    campos.append(l['from_name'])
                except:
                    continue

cols = sorted(set(campos))
cols

['apoderado_1',
 'apoderado_2',
 'apoderado_3',
 'apoderado_4',
 'apoderado_5',
 'boleta_inscripcion_bb',
 'completed_by',
 'cuadro_accionario_bb',
 'datos_inscripcion',
 'denominacion_social',
 'documento',
 'duracion_sociedad',
 'fecha_inscripcion',
 'fecha_otorgamiento',
 'id',
 'instrumento_publico',
 'is_primera_pagina',
 'lugar_otorgamiento',
 'notaria_num',
 'notario',
 'orden_del_dia_bb',
 'pagina',
 'regimen_social',
 'registro_publico_correspondiente',
 'tipos_facultades']

In [44]:
df_truth = pd.DataFrame(columns = cols)

In [45]:
for f in files:
    df_tmp = pd.DataFrame(columns = cols)
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        #print(len(ground_truth))

        for i in range(len(ground_truth)):
        #     print(i)
            identity = ground_truth[i]['id']
            documento = ground_truth[i]['file_upload'].split('-')[0]
            pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
            completed_by = ground_truth[i]['annotations'][0]['completed_by']['email']

            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                df_tmp.loc[i, 'id'] = identity
                df_tmp.loc[i, 'documento'] = documento
                df_tmp.loc[i, 'pagina'] = pagina
                df_tmp.loc[i, 'completed_by'] = completed_by

                # algunos campos aparecen en "text"
                try:
                    df_tmp.loc[i, l['from_name']] = l['value']['text'][0]
                except:
                    # otros aparecen en "choices"
                    try:
                        df_tmp.loc[i, l['from_name']] = l['value']['choices'][0]
                    except:
                        # y otros en "rectanglelabels"
                        try:
                            df_tmp.loc[i, l['from_name']] = l['value']['rectanglelabels'][0]
                        except:
                            continue
    df_truth = pd.concat([df_truth, df_tmp])
                    
# ordeno por documento y página
df_truth = df_truth.sort_values(['documento', 'pagina'])

# convierto algunos campos a int
df_truth['id'] = df_truth['id'].astype(int)
df_truth['documento'] = df_truth['documento'].astype(int)
df_truth['pagina'] = df_truth['pagina'].astype(int)

# pongo id, documento y pagina al ppio:
for c in ['pagina', 'documento', 'id']:
    first_column = df_truth.pop(c)
    df_truth.insert(0, c, first_column)

print(df_truth.info())
df_truth.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 2 to 0
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   id                                233 non-null    int64 
 1   documento                         233 non-null    int64 
 2   pagina                            233 non-null    int64 
 3   apoderado_1                       43 non-null     object
 4   apoderado_2                       19 non-null     object
 5   apoderado_3                       6 non-null      object
 6   apoderado_4                       3 non-null      object
 7   apoderado_5                       1 non-null      object
 8   boleta_inscripcion_bb             33 non-null     object
 9   completed_by                      233 non-null    object
 10  cuadro_accionario_bb              47 non-null     object
 11  datos_inscripcion                 35 non-null     object
 12  denominacion_social     

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
2,8774,10227,0,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,07/08/2017,5605,acta constitutiva,Nayarit,30,Ney González Sánchez,NaN,NaN,NaN,NaN
15,8780,10227,2,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
16,8779,10227,3,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8789,10227,4,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,acta constitutiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,8783,10227,6,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion


## Distribución de páginas

In [46]:
paginas = {}

for column in df_truth.drop(['id', 'documento', 'pagina'], axis=1):
    paginas[column] = list(df_truth.loc[df_truth[column].notnull(), 'pagina'])

In [47]:
import plotly.express as px

for k in paginas.keys():
    try:
        fig = px.histogram(paginas[k], x=paginas[k], title=k, width=600, height=400)
        fig.show()
    except:
        continue

In [48]:
# cuántos documentos hay
len(pd.unique(df_truth['documento']))

42

## Busco el outlier que se vio arriba

In [49]:
#df_truth.head()

In [50]:
df_tmp = df_truth[['documento', 'pagina', 'instrumento_publico', 'completed_by']].loc[df_truth['instrumento_publico'].notnull()]
df_tmp.head()

,documento,pagina,instrumento_publico,completed_by
2,10227,0,5605,nancy.campos@konfio.mx
3,10900,1,13377,nancy.campos@konfio.mx
10,12192,2,107826,norma.lopez@konfio.mx
55,12264,0,437,norma.lopez@konfio.mx
48,12809,0,3543,norma.lopez@konfio.mx


In [51]:
df_tmp.loc[df_tmp['pagina'] > 25]

,documento,pagina,instrumento_publico,completed_by
43,17513,26,81920,federico.rabinovich@konfio.mx


## Repito los gráficos de arriba, pero con las páginas como porcentaje

In [54]:
f = files[0]
with open(mypath+f) as json_file:
    ground_truth = json.load(json_file)

ground_truth[0]

{'id': 10219,
 'annotations': [{'id': 1953,
   'completed_by': {'id': 1,
    'email': 'yohans10@gmail.com',
    'first_name': '',
    'last_name': ''},
   'result': [{'value': {'choices': ['boleta de inscripcion']},
     'id': 'BLlH1WAcYL',
     'from_name': 'is_primera_pagina',
     'to_name': 'image',
     'type': 'choices'},
    {'original_width': 1024,
     'original_height': 1321,
     'image_rotation': 0,
     'value': {'x': 4,
      'y': 4.75206611570248,
      'width': 94.4,
      'height': 65.08264462809917,
      'rotation': 0,
      'rectanglelabels': ['boleta_inscripcion']},
     'id': 'WhZIev4on8',
     'from_name': 'boleta_inscripcion_bb',
     'to_name': 'image',
     'type': 'rectanglelabels'},
    {'value': {'text': ['merida']},
     'id': 'VenH4Uc7nq',
     'from_name': 'registro_publico_correspondiente',
     'to_name': 'image',
     'type': 'textarea'},
    {'value': {'text': ['2017009414']},
     'id': 'YlsfJS8EZ7',
     'from_name': 'datos_inscripcion',
     'to_n

In [55]:
df_truth

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
2,8774,10227,0,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,07/08/2017,5605,acta constitutiva,Nayarit,30,Ney González Sánchez,NaN,NaN,NaN,NaN
15,8780,10227,2,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
16,8779,10227,3,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8789,10227,4,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,acta constitutiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,8783,10227,6,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,10226,26228,2,NaN,NaN,NaN,NaN,NaN,NaN,yohans10@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
28,10229,26228,3,NaN,NaN,NaN,NaN,NaN,NaN,yohans10@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,10236,26228,5,NaN,NaN,NaN,NaN,NaN,NaN,yohans10@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion
1,10221,26228,8,edgar guzman burgos,NaN,NaN,NaN,NaN,NaN,yohans10@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_truth.loc[df_truth['boleta_inscripcion_bb'].notnull()]

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
0,8795,10329,13,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GUADALAJARA,NaN
8,8957,12192,46,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE LA PROPIEDAD Y DE COMERCIO...,NaN
7,8987,12264,16,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE LA PROPIEDAD Y DEL COMERCI...,NaN
3,9046,12853,23,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE LA PROPIEDAD Y DEL COMERCI...,NaN
2,9057,12863,13,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,registro publico de comercio de cholula,NaN
1,9074,13061,17,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE LA PROPIEDAD Y DEL COMERCI...,NaN
0,9100,13073,0,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE COMERCIO DE MERIDA,NaN
28,9107,13697,15,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE LA PROPIEDAD Y DE COMERCIO...,NaN
23,9128,13874,34,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE COMERCIO DE LA CIUDAD DE M...,NaN
15,9189,13881,38,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,NaN,NaN,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE COMERCIO DE LA CIUDAD DE M...,NaN
